In [1]:
from transformers import AutoModelForCausalLM, GPTNeoXForCausalLM, GPTNeoXTokenizerFast
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_dataset

In [2]:
DATASET = 1
SPLIT = 'train'

data_fpath = f'/home/echeng/llm_compositionality/data/train_prompts_{DATASET}_words_correlated.txt'
val_fpath = f'/home/echeng/llm_compositionality/data/test_prompts_{DATASET}_words_correlated.txt'


In [3]:
dataset = load_dataset('text', data_files={'train': data_fpath, 'val': val_fpath})

## LOAD MODEL

In [4]:
MODEL = 'EleutherAI/pythia-410m-deduped'

In [5]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL, 
    # load_in_8bit=True
    # torch_dtype=torch.float16,
)
tokenizer = GPTNeoXTokenizerFast.from_pretrained(MODEL)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
tokenizer.pad_token = tokenizer.eos_token

In [8]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=1, remove_columns=['text'])
train_dataset = tokenized_dataset['train']
val_dataset = tokenized_dataset['val']

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
def group_texts(examples):
    # Concatenate all texts into one long string of token IDs
    concatenated = {k: list(sum(examples[k], [])) for k in examples.keys()}

    # Calculate total length and ensure it's a multiple of block_size
    total_length = len(concatenated[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size

    # Split the concatenated data into chunks of block_size
    result = {
        k: [t[i:i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    
    # Copy input_ids to labels for causal language modeling
    result["labels"] = result["input_ids"].copy()
    
    return result

# Example block size (set according to your model's requirements)
block_size = 20

# Apply the grouping function to the tokenized dataset
lm_train_dataset = train_dataset.map(group_texts, batched=True)
lm_val_dataset = val_dataset.map(group_texts, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
lm_train_dataset.set_format("torch")
lm_val_dataset.set_format("torch")

## Finetune model

In [11]:
BATCH_SIZE = 1

In [30]:
training_args = TrainingArguments(
    f"emcheng/{MODEL}-finetuned-{DATASET}-words-correlated",
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    push_to_hub=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    save_steps=0.25,
    save_strategy='steps',
    report_to=[],
    fp16=True,  # Use mixed precision training if supported by your hardware
)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_train_dataset,
    eval_dataset=lm_val_dataset,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()

Step,Training Loss
500,0.140300
1000,0.142300
1500,0.171500
2000,0.131600
2500,0.139000
3000,0.125100
3500,0.151300
4000,0.117000
4500,0.141200
5000,0.129500


In [17]:
from huggingface_hub import notebook_login

notebook_login()